<a href="https://colab.research.google.com/github/gaonkarrs/AIML/blob/main/SampleRAGBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import torch

# Sample corpus
corpus = [
    "The capital of France is Paris.",
    "Python is a popular programming language.",
    "Photosynthesis occurs in the chloroplasts of plant cells."
]

# Step 1: Initialize retriever and embed corpus
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = retriever_model.encode(corpus, convert_to_tensor=True)

# Step 2: User query
query = "Where does photosynthesis happen in plants?"
query_embedding = retriever_model.encode(query, convert_to_tensor=True)

# Step 3: Retrieve relevant document
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_result = torch.argmax(cos_scores).item()
retrieved_context = corpus[top_result]

# Step 4: Use a generative model (e.g., T5 or GPT) to generate answer
generator = pipeline("text-generation", model="gpt2")
input_prompt = f"Context: {retrieved_context}\nQuestion: {query}\nAnswer:"
response = generator(input_prompt, max_length=50, do_sample=False)[0]['generated_text']

print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: Photosynthesis occurs in the chloroplasts of plant cells.
Question: Where does photosynthesis happen in plants?
Answer: In the chloroplasts of plants, the photosynthetic activity of the chloroplasts is regulated by the


In [2]:
def evaluate_trace(context, answer, query):
    evaluation = {}

    evaluation['Utilization'] = "photosynthesis" in answer.lower() and "chloroplast" in answer.lower()
    evaluation['Relevance'] = "photosynthesis" in context.lower()
    evaluation['Adherence'] = "chloroplast" in answer.lower() and "photosynthesis" in context.lower()
    evaluation['Completeness'] = all(word in answer.lower() for word in ["photosynthesis", "chloroplast"])

    return evaluation


In [3]:
evaluation = evaluate_trace(retrieved_context, response, query)
print("TRACe Evaluation:")
for metric, result in evaluation.items():
    print(f"{metric}: {'✔️' if result else '❌'}")


TRACe Evaluation:
Utilization: ✔️
Relevance: ✔️
Adherence: ✔️
Completeness: ✔️
